In [1]:
import math
#import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
df = pd.concat(
    map(pd.read_csv, ['br1.csv','br2.csv','br3.csv','br4.csv','br5.csv','br6.csv','br7.csv','br8.csv','br9.csv',]), ignore_index=True)
df.head()

,C1,C2,C3,A1,A2,A3,A4,A5,M1,E1,...,E3,E4,E5,L1,L2,S1,S2,S3,S4,level
0,13444,2807,8643,384,11469,1934,1934.0,0,13224.0,404,...,160,6,140,24,34,23.0,21.72,1922.0,1223,1
1,39028,4771,27065,1326,36049,2665,2665.0,574,38197.5,4033,...,397,32,323,40,102,71.0,65.80,2629.5,1316,1
2,46088,12044,29959,894,42451,3275,3275.0,236,46075.5,443,...,694,30,461,60,300,183.0,183.00,3444.0,2013,1
3,14247,9878,3213,54,11997,1872,1872.0,0,14619.5,272,...,125,0,94,5,49,44.0,40.68,1850.0,318,1
4,685574,255468,270436,1026,611308,51173,51173.0,420,686488.5,133368,...,6221,150,4379,542,2041,1656.0,1293.52,50514.5,63914,1


In [4]:
df.dtypes

C1         int64
C2         int64
C3         int64
A1         int64
A2         int64
A3         int64
A4       float64
A5         int64
M1       float64
E1         int64
E2         int64
E3         int64
E4         int64
E5         int64
L1         int64
L2         int64
S1       float64
S2       float64
S3       float64
S4         int64
level      int64
dtype: object

In [6]:
df = df.astype(int)

In [7]:
df.dtypes

C1       int32
C2       int32
C3       int32
A1       int32
A2       int32
A3       int32
A4       int32
A5       int32
M1       int32
E1       int32
E2       int32
E3       int32
E4       int32
E5       int32
L1       int32
L2       int32
S1       int32
S2       int32
S3       int32
S4       int32
level    int32
dtype: object

# prep training set

In [12]:
 factors = df.iloc[:,:-1]

values = factors.values
training_data_len = math.ceil(len(values)* 0.8)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(values.reshape(-1,1))
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(1, len(train_data)):
    x_train.append(train_data[i-1:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# prep test set

In [13]:
test_data = scaled_data[training_data_len-1: , : ]
x_test = []
y_test = values[training_data_len:]

for i in range(60, len(test_data)):
  x_test.append(test_data[i-1:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# setup of lstm structure

In [14]:
model = keras.Sequential()
model.add(layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(layers.LSTM(100, return_sequences=False))
model.add(layers.Dense(25))
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 100)            40800     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 25)                2525      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 123,751
Trainable params: 123,751
Non-trainable params: 0
_________________________________________________________________


# training lstm model

In [15]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size= 1, epochs=3)

Epoch 1/3
744342/744342 [==============================] - 2533s 3ms/step - loss: 5.9174e-07
Epoch 2/3
744342/744342 [==============================] - 2658s 4ms/step - loss: 5.9017e-07
Epoch 3/3
744342/744342 [==============================] - 2468s 3ms/step - loss: 5.8874e-07


# model evaluation

In [16]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

ValueError: operands could not be broadcast together with shapes (17864158,1) (186085,20) 